# General instructions

The goal of the project is to materialize a set of **exploratory workloads** over a real-world, large-scale,  open-domain KG: [WikiData](https://www.wikidata.org/wiki/Wikidata:Main_Page)

An exploratory workload  is composed by a set of queries, where each query is related to the information obtained previously.

An exploratory workload starts with a usually vague, open ended question, and does not assume the person issuing the workload has a clear understanding of the data contained in the target database or its structure.

Remeber that:

1. All the queries must run in the python notebook
2. You can use classes and properties only if you find them via a SPARQL query that must be present in the notebook
3. You do not delete useless queries. Keep everything that is synthatically valid 

```
?p <http://schema.org/name> ?name .
```
    
is the BGP returning a human-readable name of a property or a class in Wikidata.
    
    

In [2]:
## SETUP used later

from SPARQLWrapper import SPARQLWrapper, JSON


prefixString = """
##-9b81ceb93a-##
PREFIX wd: <http://www.wikidata.org/entity/> 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> 
PREFIX sc: <http://schema.org/>
"""

# select and construct queries
def run_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-02.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
       results = sparql.query()
       json_results = results.convert()
       if len(json_results['results']['bindings'])==0:
          print("Empty")
          return 0
    
       for bindings in json_results['results']['bindings']:
          print( [ (var, value['value'])  for var, value in bindings.items() ] )

       return len(json_results['results']['bindings'])

    except Exception as e :
        print("The operation failed", e)
    
# ASk queries
def run_ask_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-02.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        return sparql.query().convert()

    except Exception as e :
        print("The operation failed", e)


# GEO Workflow Series ("archaeological sites") 

Consider the following exploratory information need:

> Search for archaeological site in the world, across countries, continents, and in reference to their culture

## Useful URIs for the current workflow


The following are given:

| IRI           | Description   | Role      |
| -----------   | -----------   |-----------|
| `wdt:P1647`   | subproperty   | predicate |
| `wdt:P31`     | instance of   | predicate |
| `wdt:P279`    | subclass      | predicate |
| `wdt:P17`     | country       | predicate |
| `wd:Q38`      | Italy  | node      |
| `wd:Q641556`  | Verona Arena  | node      |
| `wd:Q1747689` | Ancient Rome  | node |
| `wd:Q46`      | Europe        | node |
| `wd:Q173527`  | Knossos       | node |
| `wd:Q839954`  | archaeological site | node |


Also consider

```
?p wdt:P17 wd:Q38  . 
?p wdt:P31 wd:Q839954  . 
```

is the BGP to retrieve all **archaeological sites in italy**

## Workload Goals

1. Identify the BGP that connect an archaeological site to the country, the continent, and the culture 

2. Identify the BGP to retrieve other types of an archaeological site, e.g., human settlement or theatre

3. Is there any relevant numerical attribute that describes these sites, e.g., number visitors?

4. Analyze the number of archaeological sites per type and country
 
   4.1 Which country has more archaeological sites? Which country has the most  human settlements?
   
   4.2 Which countries have Ancient Rome sites, which other "archaeological cultures" are described?
   
   4.3 Which country has the most diverse set of civilizations or cultures across its sites?
   
   4.4 If you are interested in visiting some sites, which country would you pick? Based on what criteria?


In [6]:
# start your workflow here

In [3]:
queryString = """
SELECT COUNT( ?p)
WHERE { 
?p wdt:P17 wd:Q38  . 
?p wdt:P31 wd:Q839954  . 

} 

"""

print("Results")
run_query(queryString)

Results
[('callret-0', '1869')]


1

 **1 - Identify the BGP that connect an archaeological site to the country, the continent, and the culture**

Proprerty of the italian sites

In [12]:
queryString = """
SELECT DISTINCT ?p ?pName
WHERE { 

?site wdt:P17 wd:Q38  . 
?site wdt:P31 wd:Q839954  .
?site ?p ?obj .

OPTIONAL {?site <http://schema.org/name> ?sName}
OPTIONAL {?p <http://schema.org/name> ?pName}

} 
ORDER BY ASC (?pName)

"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.w3.org/1999/02/22-rdf-syntax-ns#type')]
[('p', 'http://www.wikidata.org/prop/direct-normalized/P646')]
[('p', 'http://www.wikidata.org/prop/direct-normalized/P2671')]
[('p', 'http://www.wikidata.org/prop/direct-normalized/P1584')]
[('p', 'http://www.wikidata.org/prop/direct-normalized/P1936')]
[('p', 'http://www.wikidata.org/prop/direct-normalized/P214')]
[('p', 'http://www.wikidata.org/prop/direct-normalized/P227')]
[('p', 'http://www.wikidata.org/prop/direct-normalized/P1566')]
[('p', 'http://www.wikidata.org/prop/direct-normalized/P244')]
[('p', 'http://www.wikidata.org/prop/direct-normalized/P5782')]
[('p', 'http://www.wikidata.org/prop/direct-normalized/P2950')]
[('p', 'http://www.wikidata.org/prop/direct-normalized/P269')]
[('p', 'http://www.wikidata.org/prop/direct-normalized/P8519')]
[('p', 'http://www.wikidata.org/prop/direct-normalized/P5063')]
[('p', 'http://www.wikidata.org/prop/direct-normalized/P1667')]
[('p', 'http://www.wikidata.org/prop/direc

333

"Country" is P17; P131 'located in the administrative territorial entity', P30 is "continent"; P2596  "culture". Now I try to found this information on a specific site. 

In [16]:
queryString = """
SELECT DISTINCT ?site ?sName
WHERE { 

?site wdt:P17 wd:Q38  . 
?site wdt:P31 wd:Q839954  .
?site ?p ?obj .

OPTIONAL {?site <http://schema.org/name> ?sName}
OPTIONAL {?p <http://schema.org/name> ?pName}

} 
ORDER BY ASC (?sName)
LIMIT 500

"""

print("Results")
run_query(queryString)

Results
[('site', 'http://www.wikidata.org/entity/Q37300520'), ('sName', '"Riparo Ranaldi" prehistoric site')]
[('site', 'http://www.wikidata.org/entity/Q1641527'), ('sName', 'Abbazia di Santa Maria di Corazzo')]
[('site', 'http://www.wikidata.org/entity/Q29905385'), ('sName', 'Abella')]
[('site', 'http://www.wikidata.org/entity/Q318664'), ('sName', 'Abellinum')]
[('site', 'http://www.wikidata.org/entity/Q3471777'), ('sName', 'Abini nuragic sanctuary')]
[('site', 'http://www.wikidata.org/entity/Q3895459'), ('sName', 'Accesa Lake Archaeological Park')]
[('site', 'http://www.wikidata.org/entity/Q85429625'), ('sName', 'Accu Sa Pira Nuraghe')]
[('site', 'http://www.wikidata.org/entity/Q3604250'), ('sName', 'Acerenthia')]
[('site', 'http://www.wikidata.org/entity/Q630709'), ('sName', 'Acerrae')]
[('site', 'http://www.wikidata.org/entity/Q105394681'), ('sName', 'Acqua Acetosa')]
[('site', 'http://www.wikidata.org/entity/Q56258133'), ('sName', 'Acqua Fredda')]
[('site', 'http://www.wikidata.o

500

Exploring DOMUS AUREA. Q622438 Domus Aurea

In [18]:
queryString = """
SELECT DISTINCT ?cName ?conName ?locName ?culName
WHERE { 

wd:Q622438 wdt:P17 ?country ;
         wdt:P131 ?location  ;
         wdt:P30 ?continent ;
         wdt:P2596 ?culture .

OPTIONAL{?country <http://schema.org/name> ?cName}
OPTIONAL{?location <http://schema.org/name> ?locName}
OPTIONAL{?continent <http://schema.org/name> ?conName}
OPTIONAL{?culture <http://schema.org/name> ?culName}

} 


"""

print("Results")
run_query(queryString)

Results
Empty


0

Maybe it does not have all the properties.

In [21]:
queryString = """
SELECT DISTINCT ?country ?cName ?conName ?locName ?culName
WHERE { 

wd:Q622438 wdt:P17 ?country .

      OPTIONAL{wd:Q622438 wdt:P131 ?location  .
                  ?location <http://schema.org/name> ?locName}
      OPTIONAL{wd:Q622438 wdt:P30 ?continent .
                  ?continent <http://schema.org/name> ?conName}
      OPTIONAL{wd:Q622438  wdt:P2596 ?culture .
                  ?culture <http://schema.org/name> ?culName}
        

?country <http://schema.org/name> ?cName




} 


"""

print("Results")
run_query(queryString)

Results
[('country', 'http://www.wikidata.org/entity/Q38'), ('cName', 'Italy'), ('locName', 'Rome'), ('culName', 'Ancient Rome')]
[('country', 'http://www.wikidata.org/entity/Q38'), ('cName', 'Italy'), ('locName', 'Monti'), ('culName', 'Ancient Rome')]


2

search the Continent of Italy

In [23]:
queryString = """
SELECT DISTINCT ?p ?pName
WHERE { 

wd:Q38 ?p ?obj .

?p <http://schema.org/name> ?pName
} 
ORDER BY ASC(?pName)
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P8061'), ('pName', 'AGROVOC ID')]
[('p', 'http://www.wikidata.org/prop/direct/P5198'), ('pName', 'ASC Leiden Thesaurus ID')]
[('p', 'http://www.wikidata.org/prop/direct/P6150'), ('pName', 'Academy Awards Database nominee ID')]
[('p', 'http://www.wikidata.org/prop/direct/P8895'), ('pName', 'All the Tropes identifier')]
[('p', 'http://www.wikidata.org/prop/direct/P7870'), ('pName', 'Analysis & Policy Observatory term ID')]
[('p', 'http://www.wikidata.org/prop/direct/P8785'), ('pName', 'AniDB tag ID')]
[('p', 'http://www.wikidata.org/prop/direct/P9629'), ('pName', 'Armeniapedia ID')]
[('p', 'http://www.wikidata.org/prop/direct/P6200'), ('pName', 'BBC News topic ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1617'), ('pName', 'BBC Things ID')]
[('p', 'http://www.wikidata.org/prop/direct/P9037'), ('pName', 'BHCL UUID')]
[('p', 'http://www.wikidata.org/prop/direct/P2581'), ('pName', 'BabelNet ID')]
[('p', 'http://www.wikidata.org/prop/d

230

P30 is "continent"

In [24]:
queryString = """
SELECT DISTINCT ?obj ?objName
WHERE { 

wd:Q38 wdt:P30 ?obj .

?obj <http://schema.org/name> ?objName
} 

"""

print("Results")
run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q46'), ('objName', 'Europe')]


1


Fixing the query on the Domus Aurea

In [47]:
queryString = """
SELECT DISTINCT ?country ?cName ?conName ?locName ?culture ?culName
WHERE { 

wd:Q622438 wdt:P17 ?country .
?country wdt:P30 ?continent .
                ?continent <http://schema.org/name> ?conName .

wd:Q622438  wdt:P2596 ?culture .
                  ?culture <http://schema.org/name> ?culName .
        

?country <http://schema.org/name> ?cName .

} 


"""

print("Results")
run_query(queryString)

Results
[('country', 'http://www.wikidata.org/entity/Q38'), ('cName', 'Italy'), ('conName', 'Europe'), ('culture', 'http://www.wikidata.org/entity/Q1747689'), ('culName', 'Ancient Rome')]


1

**Answer 1**

The BGP that connect an archaeological site to the country, the continent, and the culture is 

```
?site wdt:P31 wd:Q839954  .
?site wdt:P17 ?country .
?country wdt:P30 ?continent .
                ?continent <http://schema.org/name> ?conName .

?site  wdt:P2596 ?culture .
                  ?culture <http://schema.org/name> ?culName .
        

?country <http://schema.org/name> ?cName .
?site <http://schema.org/name> ?sName .

```

--------------

**2 - Identify the BGP to retrieve other types of an archaeological site, e.g., human settlement or theatre**

I've discovered some properties

P527 'has part' <br>
P2670 'has parts of the class' <br>
P361 'part of' <br>
P279 'subclass of' <br>
P910 "topic's main category" <br>

Looking of the properties on Domus Aurea

In [27]:
queryString = """
SELECT DISTINCT ?p1 ?p2 ?p3 ?p4 ?p5
WHERE { 


wd:Q622438 wdt:P527 ?p1 .
OPTIONAL{?p1 <http://schema.org/name> ?p1Name .}

OPTIONAL {wd:Q622438 wdt:P2670 ?p2 .
?p2 <http://schema.org/name> ?p2Name .}
OPTIONAL {wd:Q622438  wdt:P361 ?p3 .
?p3 <http://schema.org/name> ?p3Name .}
OPTIONAL {wd:Q622438   wdt:P279 ?p4 .
?p4 <http://schema.org/name> ?p4Name .}
OPTIONAL {wd:Q622438   wdt:P910 ?p5 .
?p5 <http://schema.org/name> ?p5Name .}
            
            
} 


"""

print("Results")
run_query(queryString)

Results
Empty


0

Using a generic site

In [28]:
queryString = """
SELECT DISTINCT ?p1 ?p2 ?p3 ?p4 ?p5
WHERE { 

?site wdt:P31 wd:Q839954  ;
        wdt:P527 ?p1 ;
        wdt:P2670 ?p2 ;
        wdt:P361 ?p3 ;
        wdt:P279 ?p4 ;
        wdt:P910 ?p5 .       
        
OPTIONAL{?p1 <http://schema.org/name> ?p1Name .}
OPTIONAL {?p2 <http://schema.org/name> ?p2Name .}
OPTIONAL {?p3 <http://schema.org/name> ?p3Name .}
OPTIONAL {?p4 <http://schema.org/name> ?p4Name .}
OPTIONAL {?p5 <http://schema.org/name> ?p5Name .}
            
            
} 


"""

print("Results")
run_query(queryString)

Results
Empty


0

One property at a time

In [32]:
queryString = """
SELECT DISTINCT ?p1 ?p1Name
WHERE { 

?site wdt:P31 wd:Q839954  ;
        wdt:P527 ?p1 .     ## hasPart
        
OPTIONAL {?p1 <http://schema.org/name> ?p1Name .}
           
            
} 


"""

print("Results")
run_query(queryString)

Results
[('p1', 'http://www.wikidata.org/entity/Q98482636'), ('p1Name', 'Brunnen')]
[('p1', 'http://www.wikidata.org/entity/Q98483067'), ('p1Name', 'Einfriedung')]
[('p1', 'http://www.wikidata.org/entity/Q17529600'), ('p1Name', 'Remains Of Roman Town House')]
[('p1', 'http://www.wikidata.org/entity/Q17647175'), ('p1Name', 'Pair of bowl barrows 405m east of East Water Drove (Part of Priddy Nine Barrows Cemetery)')]
[('p1', 'http://www.wikidata.org/entity/Q26435803'), ('p1Name', 'Wheelhead cross at Longstone')]
[('p1', 'http://www.wikidata.org/entity/Q26435804'), ('p1Name', 'Guidepost at Longstone')]
[('p1', 'http://www.wikidata.org/entity/Q31053052'), ('p1Name', 'Sharpenhoe Clappers')]
[('p1', 'http://www.wikidata.org/entity/Q98683009'), ('p1Name', 'Burg')]
[('p1', 'http://www.wikidata.org/entity/Q43176876'), ('p1Name', 'Britus I')]
[('p1', 'http://www.wikidata.org/entity/Q43176887'), ('p1Name', 'Britus II')]
[('p1', 'http://www.wikidata.org/entity/Q56649684'), ('p1Name', 'Britus III')]

2756

In [33]:
queryString = """
SELECT DISTINCT ?p1 ?p1Name
WHERE { 

?site wdt:P31 wd:Q839954  ;
        wdt:P361 ?p1 .     ##partOf
        
OPTIONAL {?p1 <http://schema.org/name> ?p1Name .}
           
            
} 
LIMIT 40

"""

print("Results")
run_query(queryString)

Results
[('p1', 'http://www.wikidata.org/entity/Q14546501'), ('p1Name', 'list of cultural properties in Oberlunkhofen')]
[('p1', 'http://www.wikidata.org/entity/Q7362097'), ('p1Name', 'Roman Museum')]
[('p1', 'http://www.wikidata.org/entity/Q64822163'), ('p1Name', 'Ach Valley')]
[('p1', 'http://www.wikidata.org/entity/Q22026170'), ('p1Name', 'Cathedral and Churches of Echmiatsin and the Archaeological Site of Zvartnots')]
[('p1', 'http://www.wikidata.org/entity/Q89272148'), ('p1Name', 'Barbegal aqueduct and mill')]
[('p1', 'http://www.wikidata.org/entity/Q27048585'), ('p1Name', 'Pompeii Regio I Insula 8')]
[('p1', 'http://www.wikidata.org/entity/Q3931883'), ('p1Name', 'region I of the archaeological excavations of Pompeii')]
[('p1', 'http://www.wikidata.org/entity/Q4923872'), ('p1Name', 'Nubian Monuments from Abu Simbel to Philae')]
[('p1', 'http://www.wikidata.org/entity/Q523026'), ('p1Name', 'Old Rome')]
[('p1', 'http://www.wikidata.org/entity/Q55678739'), ('p1Name', 'Medma - Rosarno

40

In [36]:
queryString = """
SELECT DISTINCT ?p1 ?p1Name
WHERE { 

?site wdt:P31 wd:Q839954  ;
        wdt:P279 ?p1 .     ##subClassOf
        
OPTIONAL {?p1 <http://schema.org/name> ?p1Name .}
           
            
} 
LIMIT 60

"""

print("Results")
run_query(queryString)

Results
[('p1', 'http://www.wikidata.org/entity/Q30504597'), ('p1Name', 'Maya civilization site')]
[('p1', 'http://www.wikidata.org/entity/Q96376027'), ('p1Name', 'former synagogue')]
[('p1', 'http://www.wikidata.org/entity/Q96352513'), ('p1Name', 'religious building ruin')]
[('p1', 'http://www.wikidata.org/entity/Q96371565'), ('p1Name', 'former monastery')]
[('p1', 'http://www.wikidata.org/entity/Q210272'), ('p1Name', 'cultural heritage')]
[('p1', 'http://www.wikidata.org/entity/Q259209'), ('p1Name', 'industrial archaeology')]
[('p1', 'http://www.wikidata.org/entity/Q22173125')]
[('p1', 'http://www.wikidata.org/entity/Q16560'), ('p1Name', 'palace')]
[('p1', 'http://www.wikidata.org/entity/Q1935728'), ('p1Name', 'stone circle')]
[('p1', 'http://www.wikidata.org/entity/Q101659'), ('p1Name', 'dolmen')]
[('p1', 'http://www.wikidata.org/entity/Q64526263')]
[('p1', 'http://www.wikidata.org/entity/Q23413'), ('p1Name', 'castle')]
[('p1', 'http://www.wikidata.org/entity/Q839954'), ('p1Name', '

24

In [35]:
queryString = """
SELECT DISTINCT ?p1 ?p1Name
WHERE { 

?site wdt:P31 wd:Q839954  ;
        wdt:P910 ?p1 .     ##topic's main category
        
OPTIONAL {?p1 <http://schema.org/name> ?p1Name .}
           
            
} 
LIMIT 40

"""

print("Results")
run_query(queryString)

Results
[('p1', 'http://www.wikidata.org/entity/Q25216144'), ('p1Name', 'Category:Keros')]
[('p1', 'http://www.wikidata.org/entity/Q15723537'), ('p1Name', 'Category:Ancient Greek theatre (Ohrid)')]
[('p1', 'http://www.wikidata.org/entity/Q7276695'), ('p1Name', 'Category:Copán')]
[('p1', 'http://www.wikidata.org/entity/Q55272885'), ('p1Name', 'Category:Cadzow Castle')]
[('p1', 'http://www.wikidata.org/entity/Q93128474'), ('p1Name', "Category:Wayland's Smithy")]
[('p1', 'http://www.wikidata.org/entity/Q10014734'), ('p1Name', 'Category:Temple of Zeus in Olympia')]
[('p1', 'http://www.wikidata.org/entity/Q106231688'), ('p1Name', 'Category:West Kennet Long Barrow')]
[('p1', 'http://www.wikidata.org/entity/Q18642948')]
[('p1', 'http://www.wikidata.org/entity/Q9507876'), ('p1Name', 'Category:Erlitou site')]
[('p1', 'http://www.wikidata.org/entity/Q21037500'), ('p1Name', 'Category:Palace of Domitian')]
[('p1', 'http://www.wikidata.org/entity/Q55288737'), ('p1Name', 'Category:Erechtheum')]
[('p

40

Maybe i can look to the property P1083 "maximum capacity"

In [37]:
queryString = """
SELECT DISTINCT ?site ?sName ?obj ?objName
WHERE { 

?site wdt:P17 wd:Q38  . 
?site wdt:P31 wd:Q839954  .
?site wdt:P1083 ?obj .

OPTIONAL {?site <http://schema.org/name> ?sName}
OPTIONAL {?obj <http://schema.org/name> ?objName}

} 
ORDER BY ASC (?sName)

"""

print("Results")
run_query(queryString)

Results
[('site', 'http://www.wikidata.org/entity/Q10285'), ('sName', 'Colosseum'), ('obj', '65000')]


1

Exploring Colosseo

In [38]:
queryString = """
SELECT DISTINCT ?p ?pName ?obj ?objName
WHERE { 

wd:Q10285 ?p ?obj .

OPTIONAL {?p <http://schema.org/name> ?pName}
OPTIONAL {?obj <http://schema.org/name> ?objName}

} 
ORDER BY ASC (?pName)

"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), ('obj', 'http://wikiba.se/ontology#Item')]
[('p', 'http://www.wikidata.org/prop/direct-normalized/P1566'), ('obj', 'http://sws.geonames.org/6269248/')]
[('p', 'http://www.wikidata.org/prop/direct-normalized/P1584'), ('obj', 'https://pleiades.stoa.org/places/285857974')]
[('p', 'http://www.wikidata.org/prop/direct-normalized/P1936'), ('obj', 'https://imperium.ahlfeldt.se/places/23770')]
[('p', 'http://www.wikidata.org/prop/direct-normalized/P214'), ('obj', 'http://viaf.org/viaf/134678625')]
[('p', 'http://www.wikidata.org/prop/direct-normalized/P214'), ('obj', 'http://viaf.org/viaf/167878600')]
[('p', 'http://www.wikidata.org/prop/direct-normalized/P214'), ('obj', 'http://viaf.org/viaf/295358893')]
[('p', 'http://www.wikidata.org/prop/direct-normalized/P214'), ('obj', 'http://viaf.org/viaf/400154387233230970007')]
[('p', 'http://www.wikidata.org/prop/direct-normalized/P227'), ('obj', 'https://d-nb.info/gnd/4258895-9')]
[

123

Can I use P31 to find other types of sites?

In [46]:
queryString = """
SELECT DISTINCT ?obj ?objName
WHERE { 

?site wdt:P17 wd:Q38  . 
?site wdt:P31 wd:Q839954  .
?site wdt:P31 ?obj .

OPTIONAL {?site <http://schema.org/name> ?sName}
OPTIONAL {?obj <http://schema.org/name> ?objName}

} 
ORDER BY ASC (?objName)

"""

print("Results")
run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q1243603'), ('objName', '14 regions of Augustan Rome')]
[('obj', 'http://www.wikidata.org/entity/Q93342462'), ('objName', 'Ancient Greek archaeological site')]
[('obj', 'http://www.wikidata.org/entity/Q1161819'), ('objName', 'Ancient Roman unit of measurement')]
[('obj', 'http://www.wikidata.org/entity/Q1114895'), ('objName', 'Comitium')]
[('obj', 'http://www.wikidata.org/entity/Q782970'), ('objName', 'Domus')]
[('obj', 'http://www.wikidata.org/entity/Q20010053'), ('objName', 'Etruscan grave')]
[('obj', 'http://www.wikidata.org/entity/Q1371688'), ('objName', 'Etruscan temple')]
[('obj', 'http://www.wikidata.org/entity/Q69391739'), ('objName', 'Greek colony')]
[('obj', 'http://www.wikidata.org/entity/Q2860319'), ('objName', 'Greek theatre')]
[('obj', 'http://www.wikidata.org/entity/Q1057440'), ('objName', 'Heraion')]
[('obj', 'http://www.wikidata.org/entity/Q279616'), ('objName', 'Horreum')]
[('obj', 'http://www.wikidata.org/entity/Q38956

210

**Answer 2**

From the useful information found before

To fine the specific site I can surf the above list, and then substitute the entity to the "obj"

```
?site wdt:P17 wd:Q38  . 
?site wdt:P31 wd:Q839954  .
?site wdt:P31 ?obj .

OPTIONAL {?site <http://schema.org/name> ?sName}
OPTIONAL {?obj <http://schema.org/name> ?objName}

```

--------------

**3 - Is there any relevant numerical attribute that describes these sites, e.g., number visitors?**

YES! There are multiple numerical values:
P2048 "height" <br>
P2043 "length"<br>
P1083 "maximum capacity" <br>
P1082 "population" <br>
P2324 "quantity buried" <br>
P1174 "visitors per year" <br>
P2049 "width" <br>

```

?site wdt:P17 wd:Q38  . 
?site wdt:P31 wd:Q839954  .
?site *specific property from above* ?obj .

OPTIONAL {?site <http://schema.org/name> ?sName}
OPTIONAL {?obj <http://schema.org/name> ?objName}

```

---------------

**4 - Which country has more archaeological sites? Which country has the most human settlements?**

Using previous query, removing the constrint on "Italy" Q38

In [5]:
queryString = """
SELECT ?country ?cName (COUNT(?site) AS ?num_archSites) 
WHERE { 

?site wdt:P17 ?country  . 
?site wdt:P31 wd:Q839954  .


OPTIONAL {?site <http://schema.org/name> ?sName}
OPTIONAL {?country <http://schema.org/name> ?cName}

} 
GROUP BY (?num_archSites) (?country)(?cName)
ORDER BY DESC (?num_archSites)
LIMIT 10

"""

print("Results")
run_query(queryString)

Results
[('country', 'http://www.wikidata.org/entity/Q145'), ('cName', 'United Kingdom'), ('num_archSites', '7026')]
[('country', 'http://www.wikidata.org/entity/Q29'), ('cName', 'Spain'), ('num_archSites', '6168')]
[('country', 'http://www.wikidata.org/entity/Q221'), ('cName', 'North Macedonia'), ('num_archSites', '4329')]
[('country', 'http://www.wikidata.org/entity/Q183'), ('cName', 'Germany'), ('num_archSites', '2825')]
[('country', 'http://www.wikidata.org/entity/Q217'), ('cName', 'Moldova'), ('num_archSites', '2674')]
[('country', 'http://www.wikidata.org/entity/Q30'), ('cName', 'United States of America'), ('num_archSites', '2159')]
[('country', 'http://www.wikidata.org/entity/Q414'), ('cName', 'Argentina'), ('num_archSites', '2073')]
[('country', 'http://www.wikidata.org/entity/Q38'), ('cName', 'Italy'), ('num_archSites', '1869')]
[('country', 'http://www.wikidata.org/entity/Q39'), ('cName', 'Switzerland'), ('num_archSites', '984')]
[('country', 'http://www.wikidata.org/entity/

10

Ora cerco quello con più insediamenti 'Q486972'

In [7]:
queryString = """
SELECT ?country ?cName (COUNT(?site) AS ?num_Human_archSites) 
WHERE { 

?site wdt:P17 ?country  . 
?site wdt:P31 wd:Q486972  .


OPTIONAL {?site <http://schema.org/name> ?sName}
OPTIONAL {?country <http://schema.org/name> ?cName}

} 
GROUP BY (?country)(?cName)
ORDER BY DESC (?num_Human_archSites)
LIMIT 10

"""

print("Results")
run_query(queryString)

Results
[('country', 'http://www.wikidata.org/entity/Q96'), ('cName', 'Mexico'), ('num_Human_archSites', '170484')]
[('country', 'http://www.wikidata.org/entity/Q805'), ('cName', 'Yemen'), ('num_Human_archSites', '35599')]
[('country', 'http://www.wikidata.org/entity/Q38'), ('cName', 'Italy'), ('num_Human_archSites', '29596')]
[('country', 'http://www.wikidata.org/entity/Q183'), ('cName', 'Germany'), ('num_Human_archSites', '26743')]
[('country', 'http://www.wikidata.org/entity/Q668'), ('cName', 'India'), ('num_Human_archSites', '26260')]
[('country', 'http://www.wikidata.org/entity/Q836'), ('cName', 'Myanmar'), ('num_Human_archSites', '22007')]
[('country', 'http://www.wikidata.org/entity/Q159'), ('cName', 'Russia'), ('num_Human_archSites', '14743')]
[('country', 'http://www.wikidata.org/entity/Q29'), ('cName', 'Spain'), ('num_Human_archSites', '12487')]
[('country', 'http://www.wikidata.org/entity/Q794'), ('cName', 'Iran'), ('num_Human_archSites', '11166')]
[('country', 'http://www.w

10

**Answer 3**

the country with most archeological sites is UK, the one with most human settlements is Mexico

----------

**4.1 - Which countries have Ancient Rome sites, which other "archaeological cultures" are described?**

Using Domus Aurea and exploring its culture to retieve the Ancient Rome

In [8]:
queryString = """
SELECT DISTINCT ?country ?cName ?conName ?locName ?culture ?culName
WHERE { 

wd:Q622438 wdt:P17 ?country .
?country wdt:P30 ?continent .
                ?continent <http://schema.org/name> ?conName .

wd:Q622438  wdt:P2596 ?culture .
                  ?culture <http://schema.org/name> ?culName .
        

?country <http://schema.org/name> ?cName .

} 


"""

print("Results")
run_query(queryString)

Results
[('country', 'http://www.wikidata.org/entity/Q38'), ('cName', 'Italy'), ('conName', 'Europe'), ('culture', 'http://www.wikidata.org/entity/Q1747689'), ('culName', 'Ancient Rome')]


1

Exploring the countries with "Ancient Rome" Q1747689 connection

In [51]:
queryString = """
SELECT DISTINCT ?country ?cName
WHERE { 

?site wdt:P31 wd:Q839954  .
?site wdt:P17 ?country .
?country wdt:P30 ?continent .
                ?continent <http://schema.org/name> ?conName .

?site  wdt:P2596 wd:Q1747689 . ##Ancient Rome       

?country <http://schema.org/name> ?cName .

} 
ORDER BY ASC(?cName)

"""

print("Results")
run_query(queryString)

Results
[('country', 'http://www.wikidata.org/entity/Q222'), ('cName', 'Albania')]
[('country', 'http://www.wikidata.org/entity/Q262'), ('cName', 'Algeria')]
[('country', 'http://www.wikidata.org/entity/Q1747689'), ('cName', 'Ancient Rome')]
[('country', 'http://www.wikidata.org/entity/Q40'), ('cName', 'Austria')]
[('country', 'http://www.wikidata.org/entity/Q31'), ('cName', 'Belgium')]
[('country', 'http://www.wikidata.org/entity/Q219'), ('cName', 'Bulgaria')]
[('country', 'http://www.wikidata.org/entity/Q224'), ('cName', 'Croatia')]
[('country', 'http://www.wikidata.org/entity/Q79'), ('cName', 'Egypt')]
[('country', 'http://www.wikidata.org/entity/Q142'), ('cName', 'France')]
[('country', 'http://www.wikidata.org/entity/Q230'), ('cName', 'Georgia')]
[('country', 'http://www.wikidata.org/entity/Q183'), ('cName', 'Germany')]
[('country', 'http://www.wikidata.org/entity/Q41'), ('cName', 'Greece')]
[('country', 'http://www.wikidata.org/entity/Q801'), ('cName', 'Israel')]
[('country', 'ht

27

**I've found the countries with Ancient Roman sites.**

Are there other cultures? Retrieving the complete list of cultures for each country

In [55]:
queryString = """
SELECT ?cName (GROUP_CONCAT(DISTINCT ?culName;separator=", ") AS ?cultures)
WHERE { 

?site wdt:P31 wd:Q839954  .
?site wdt:P17 ?country .
?country wdt:P30 ?continent .
                ?continent <http://schema.org/name> ?conName .

?site  wdt:P2596 wd:Q1747689 . ##Ancient Rome 
?site  wdt:P2596 ?culture .

?country <http://schema.org/name> ?cName .
?culture <http://schema.org/name> ?culName .

} 
GROUP BY (?cName)
ORDER BY ASC(?cName)

"""

print("Results")
run_query(queryString)

Results
[('cName', 'Albania'), ('cultures', 'Ancient Rome')]
[('cName', 'Algeria'), ('cultures', 'Ancient Rome')]
[('cName', 'Ancient Rome'), ('cultures', 'Ancient Rome')]
[('cName', 'Austria'), ('cultures', 'Ancient Rome')]
[('cName', 'Belgium'), ('cultures', 'Ancient Rome')]
[('cName', 'Bulgaria'), ('cultures', 'Ancient Rome')]
[('cName', 'Croatia'), ('cultures', 'Ancient Rome')]
[('cName', 'Egypt'), ('cultures', 'Ancient Rome')]
[('cName', 'France'), ('cultures', 'Ancient Rome')]
[('cName', 'Georgia'), ('cultures', 'Ancient Rome')]
[('cName', 'Germany'), ('cultures', 'Ancient Rome, Germanic peoples')]
[('cName', 'Greece'), ('cultures', 'Ancient Rome')]
[('cName', 'Israel'), ('cultures', 'Ancient Rome')]
[('cName', 'Italy'), ('cultures', 'Adriatic Veneti, Ancient Greece, Ancient Rome, culture of ancient Rome')]
[('cName', 'Jordan'), ('cultures', 'Ancient Greece, Ancient Rome, Byzantine Empire')]
[('cName', 'Libya'), ('cultures', 'Ancient Rome')]
[('cName', 'Luxembourg'), ('cultures',

27

**Answer 4.2**

I've found all the answers to the questions

**4.3 Which country has the most diverse set of civilizations or cultures across its sites?**

In [56]:
queryString = """
SELECT ?cName (GROUP_CONCAT(DISTINCT ?culName;separator=", ") AS ?cultures)
WHERE { 

?site wdt:P31 wd:Q839954  .
?site wdt:P17 ?country .
?country wdt:P30 ?continent .
                ?continent <http://schema.org/name> ?conName .

?site  wdt:P2596 ?culture .

?country <http://schema.org/name> ?cName .
?culture <http://schema.org/name> ?culName .

} 
GROUP BY (?cName)
ORDER BY ASC(?cName)

"""

print("Results")
run_query(queryString)

Results
[('cName', 'Achaemenid Empire'), ('cultures', 'Babylonia, Greater Iran')]
[('cName', 'Afghanistan'), ('cultures', 'Greco-Buddhism, Hellenistic period, Indo-Sassanids')]
[('cName', 'Albania'), ('cultures', 'Ancient Rome, Illyrians')]
[('cName', 'Algeria'), ('cultures', 'Ancient Libya, Ancient Rome, Carthage, Neolithic, Phoenicians, Roman Empire')]
[('cName', 'Ancient Egypt'), ('cultures', 'Ancient Egypt')]
[('cName', 'Ancient Greece'), ('cultures', 'Ancient Greece')]
[('cName', 'Ancient Rome'), ('cultures', 'Ancient Rome, Carthage, Castro culture, Celtiberians, Roman Empire')]
[('cName', 'Argentina'), ('cultures', 'Guaraní people')]
[('cName', 'Armenia'), ('cultures', 'Kura–Araxes culture')]
[('cName', 'Assyrian Empire'), ('cultures', 'Assyrian Empire')]
[('cName', 'Austria'), ('cultures', 'Ancient Rome')]
[('cName', 'Azerbaijan'), ('cultures', 'Armenia')]
[('cName', 'Belgium'), ('cultures', 'Ancient Rome, Michelsberg culture')]
[('cName', 'Belize'), ('cultures', 'Maya civilizat

93

In [61]:
queryString = """
SELECT ?cName (COUNT(DISTINCT ?culture) AS ?num_of_cultures)
WHERE { 

?site wdt:P31 wd:Q839954  .
?site wdt:P17 ?country .
?country wdt:P30 ?continent .
                ?continent <http://schema.org/name> ?conName .

?site  wdt:P2596 ?culture .

?country <http://schema.org/name> ?cName .
?culture <http://schema.org/name> ?culName .

} 
GROUP BY (?cName)(?cultures)
ORDER BY DESC(?num_of_cultures)
LIMIT 10

"""

print("Results")
run_query(queryString)

Results
[('cName', 'Italy'), ('num_of_cultures', '61')]
[('cName', 'United States of America'), ('num_of_cultures', '46')]
[('cName', 'Israel'), ('num_of_cultures', '16')]
[('cName', 'Syria'), ('num_of_cultures', '15')]
[('cName', 'Turkey'), ('num_of_cultures', '15')]
[('cName', 'Iraq'), ('num_of_cultures', '14')]
[('cName', 'Greece'), ('num_of_cultures', '14')]
[('cName', 'Spain'), ('num_of_cultures', '14')]
[('cName', 'Germany'), ('num_of_cultures', '14')]
[('cName', 'Peru'), ('num_of_cultures', '13')]


10

**Answer 4.3** 

Italy has the highest number of different cultures.

**4.4 If you are interested in visiting some sites, which country would you pick? Based on what criteria?**

I would choose countries like Turkey or Israel. I'm curious about the Roman domination there. When I was younger I partially visited some of those sites and I remained impressed about the visible high number of different cultures present. 